### Order planning in maintenance with Google OR-Tools

In [73]:
import os
import csv
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#### Get Data

In [74]:
script_dir = os.path.abspath('')
folder_data_for_comparison = os.path.join(script_dir, "data\data_for_comparison")
files = os.listdir(folder_data_for_comparison)

with open(os.path.join(folder_data_for_comparison, 'solomon-25-c101-time_matrix.csv'), 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    time_matrix = []
    for line in csv_reader:
        time_matrix.append([int(x) for x in line])
 
with open(os.path.join(folder_data_for_comparison, 'service_times.csv'), 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    service_times = []
    for line in csv_reader:
        service_times.append([int(x) for x in line])
    service_times = service_times[0][:len(time_matrix)]

#### Functions

In [75]:
def create_data_model():
    data = {}
    data['time_matrix'] = time_matrix
    data['service_time'] = service_times
    data['num_vehicles'] = int(len(service_times) / 3)
    data['depot'] = 0
    return data

def print_solution(manager, routing, solution):
    print(f'Objective: {solution.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(manager.GetNumberOfVehicles()):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += f'{manager.IndexToNode(index)} '
            plan_output += f'Time({solution.Value(time_var)}) -> '
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += f'{manager.IndexToNode(index)} '
        plan_output += f'Time({solution.Value(time_var)})\n'
        plan_output += f'Time of the route: {solution.Value(time_var)}min\n'
        print(plan_output)
        total_time += solution.Value(time_var)
    print(f'Total time of all routes: {total_time}min')
    return total_time

#### Main

In [76]:
data = create_data_model()

manager = pywrapcp.RoutingIndexManager(
        len(data['time_matrix']),
        data['num_vehicles'],
        data['depot'])

routing = pywrapcp.RoutingModel(manager)

#Returns the travel time + service time between the two nodes.
def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node] + data['service_time'][from_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

time = 'Time'
working_time = 420 # 7 hours = 420 min
routing.AddDimension(transit_callback_index, 0, working_time, True, time)
time_dimension = routing.GetDimensionOrDie(time)
time_dimension.SetGlobalSpanCostCoefficient(10)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)                                                                                 
search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC)
search_parameters.time_limit.FromSeconds(10)

solution = routing.SolveWithParameters(search_parameters)

if solution:
    print_solution(manager, routing, solution)
else:
    print("Solver status: ", routing.status())
    print('No solution found !')

Objective: 4963
Route for vehicle 0:
0 Time(0) -> 14 Time(39) -> 16 Time(101) -> 18 Time(142) -> 0 Time(267)
Time of the route: 267min

Route for vehicle 1:
0 Time(0) -> 5 Time(15) -> 6 Time(49) -> 4 Time(81) -> 3 Time(173) -> 0 Time(279)
Time of the route: 279min

Route for vehicle 2:
0 Time(0) -> 2 Time(20) -> 1 Time(142) -> 0 Time(250)
Time of the route: 250min

Route for vehicle 3:
0 Time(0) -> 17 Time(33) -> 19 Time(98) -> 12 Time(199) -> 0 Time(267)
Time of the route: 267min

Route for vehicle 4:
0 Time(0) -> 15 Time(36) -> 13 Time(161) -> 0 Time(281)
Time of the route: 281min

Route for vehicle 5:
0 Time(0) -> 10 Time(16) -> 11 Time(79) -> 23 Time(184) -> 0 Time(257)
Time of the route: 257min

Route for vehicle 6:
0 Time(0) -> 8 Time(18) -> 9 Time(50) -> 7 Time(174) -> 0 Time(280)
Time of the route: 280min

Route for vehicle 7:
0 Time(0) -> 20 Time(10) -> 22 Time(72) -> 24 Time(105) -> 25 Time(137) -> 21 Time(202) -> 0 Time(272)
Time of the route: 272min

Total time of all route